In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Set the dataset directory
dataset_dir = '/content/drive/My Drive/TezMotifAlgilama/MOT_DB_AUG/'

In [ ]:
# Image preprocessing
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data for validation
)

In [ ]:
# Load the training data
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # ResNet50 input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)


Found 3888 images belonging to 10 classes.


In [ ]:
# Load the validation data
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # ResNet50 input size
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 967 images belonging to 10 classes.


In [ ]:
# Verify the class indices
print("Class Indices:", train_generator.class_indices)

Class Indices: {'Harlequin Stitch': 0, 'alpine crochet': 1, 'basket weave crochet stitch': 2, 'feather stitch': 3, 'herringbone crochet': 4, 'jasmine crochet': 5, 'moss crochet': 6, 'shell crochet': 7, 'star stitch': 8, 'waffle stitch': 9}


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the ResNet50 model pre-trained on ImageNet, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False)


In [ ]:
# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [ ]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5 # You can adjust the number of epochs
)

Epoch 1/5
121/121 [==============================] - 605s 5s/step - loss: 1.8688 - accuracy: 0.4251 - val_loss: 56.7617 - val_accuracy: 0.1271
Epoch 2/5
121/121 [==============================] - 27s 224ms/step - loss: 0.7726 - accuracy: 0.7627 - val_loss: 25.2550 - val_accuracy: 0.2427
Epoch 3/5
121/121 [==============================] - 27s 221ms/step - loss: 0.4754 - accuracy: 0.8558 - val_loss: 5.9835 - val_accuracy: 0.5198
Epoch 4/5
121/121 [==============================] - 27s 221ms/step - loss: 0.2746 - accuracy: 0.9116 - val_loss: 234.3774 - val_accuracy: 0.3625
Epoch 5/5
121/121 [==============================] - 27s 221ms/step - loss: 0.2139 - accuracy: 0.9349 - val_loss: 3.0529 - val_accuracy: 0.6708


In [ ]:
model.save('/content/drive/My Drive/TezMotifAlgilama/saved_model')

# Or save the model in HDF5 format
model.save('/content/drive/My Drive/TezMotifAlgilama/modelfull.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
